In [1]:
import cv2
import numpy as np


In [3]:
def adjust_brightness(image, factor):
    # Convert to HSV to scale the V (brightness) channel
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = np.clip(v * factor, 0, 255).astype(np.uint8)
    hsv = cv2.merge([h, s, v])
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [5]:

def detect_colors(image, colors):
    # Define HSV ranges for each color
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    ranges = {
        "blue":   ([100, 150, 50], [140, 255, 255]),
        "green":  ([40, 70, 70],  [80, 255, 255]),
        "yellow": ([20, 100, 100],[30, 255, 255]),
    }
    detections = []
    for name in colors:
        if name in ranges:
            lower, upper = ranges[name]
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for cnt in contours:
                if cv2.contourArea(cnt) > 500:
                    x, y, w, h = cv2.boundingRect(cnt)
                    detections.append((name, (x, y, x + w, y + h)))
    return detections

In [7]:
def main():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Cannot open camera")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        flip = cv2.flip(frame, 1)
        bright = adjust_brightness(flip, 1.2)
        detected = detect_colors(bright, ["blue", "green", "yellow"])

        for name, (x1, y1, x2, y2) in detected:
            cv2.rectangle(bright, (x1, y1), (x2, y2), (0, 255, 0), 3)
            cv2.putText(bright, name.capitalize(), (x1, y2 + 20),
                        cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 255, 255), 2, cv2.LINE_AA)

        cv2.putText(bright, "PRESS ESC TO EXIT", (10, 30),
                    cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.imshow("color_detection_live", bright)

        if cv2.waitKey(1) == 27:  # ESC key
            break

    cap.release()
    cv2.destroyAllWindows()

In [9]:
if __name__ == "__main__":
    main()
